In [1]:
from pynq import allocate
import numpy as np
from pynq import Overlay

overlay = Overlay("blk_pass.bit")

In [2]:
datasize  = 10
print(overlay.ip_dict.keys())

dict_keys(['pack_stream_to_blk_0', 'unpack_blk_to_stream_0', 'axi_dma_0'])


In [3]:
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

InStrm = overlay.pack_stream_to_blk_0
OutStrm = overlay.unpack_blk_to_stream_0

In [14]:
InStrm.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=10)
}

In [15]:
OutStrm.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  n = Register(n=10)
}

In [6]:
InStrm.register_map.n = datasize

In [8]:
OutStrm.register_map.n = datasize

In [10]:
input_buffer = allocate(shape=(datasize,),dtype=np.float32)
output_buffer = allocate(shape=(datasize,),dtype=np.float32)

In [11]:
for i in range (datasize): input_buffer[i] = i+1
for i in range(datasize): print(input_buffer[i])

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0


In [12]:
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))
print("---")
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))

DMA Source address     : 0x0
DMA Destination address: 0x0
---
Input buffer address   : 0x18049000
Output buffer address  : 0x1804a000


In [13]:
CONTROL_REGISTER = 0x0
InStrm.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
OutStrm.write(CONTROL_REGISTER, 0x81)

In [16]:
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
dma_send.wait()
dma_recv.wait()

In [ ]:
for i in range(datasize): print(output_buffer[i])

In [ ]:
del input_buffer, output_buffer